# Import libraries and model

In [57]:
from os import listdir
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gc
# import gensim.downloader as api

import torch
import torch.nn as nn

from math import ceil
import torch.optim as optim

nltk.download('stopwords')
nltk.download('wordnet')

# Load GloVe model with Gensim's API - Twitter specific embedding
# embeddings_model = api.load("glove-twitter-200")  # 200-dimensional GloVe embeddings

#To check that T4 GPU is connected
#!nvidia-smi

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/infres/kbrowder-24/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/infres/kbrowder-24/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [58]:
# %pip install "modin[all]"
# import modin.pandas as pd

In [59]:
import pandas as pd

In [60]:
# %pip install swifter
# import swifter

In [61]:
# import os
# os.environ["MODIN_ENGINE"] = "dask"  # Options: "ray" or "dask"
# import modin.pandas as pd

In [62]:
# import dask.dataframe as pd

In [63]:
import swifter

# Data preprocessing

In [131]:
# Read all training files and concatenate them into one dataframe

#import os
#print(os.getcwd())

li = []
i = 0
for filename in listdir("train_tweets"):
    if filename != '.ipynb_checkpoints':
        print(filename)
        df = pd.read_csv("train_tweets/" + filename)
        # df.drop(columns=['Timestamp'], inplace=True)
        # drop unused column(s)
        df['MatchID'] = str(i)
        df['ID'] = str(i)+ '_' + df['PeriodID'].astype(str)
        # makes sure that the match IDs are ordered from 0,1,2... with no missing values
        i+=1
        li.append(df)
df = pd.concat(li, ignore_index=True)
#print(len(df))
df

USASlovenia2010.csv
ArgentinaBelgium72.csv
AustraliaSpain34.csv
ArgentinaGermanyFinal77.csv
AustraliaNetherlands29.csv
BelgiumSouthKorea59.csv
HondurasSwitzerland54.csv
FranceGermany70.csv
GermanyBrazil74.csv
GermanyUSA57.csv
MexicoCroatia37.csv
FranceNigeria66.csv
CameroonBrazil36.csv
NetherlandsChile35.csv
PortugalGhana58.csv
GermanyAlgeria67.csv


,ID,MatchID,PeriodID,EventType,Timestamp,Tweet
0,0_0,0,0,0,1276869000000,#USA All My Stateside Followers Stand Up And R...
1,0_0,0,0,0,1276869000000,@Lynz_89 I think the ref might have been Basil...
2,0_0,0,0,0,1276869000000,Hoping a #USA win can help ease the pain of la...
3,0_0,0,0,0,1276869000000,When does this actually start? #worldcup
4,0_0,0,0,0,1276869000000,Hanson and Roy are a proper pundit line up. #w...
...,...,...,...,...,...,...
5056045,15_169,15,169,0,1404168000000,RT @FOXSoccer: 3/4 of the #WorldCup quarterfin...
5056046,15_169,15,169,0,1404168000000,RT @Rodolph_hilal: Plz guys RETWEET .. \n\nLet...
5056047,15_169,15,169,0,1404168000000,RT @Joey7Barton: Algeria can take a lot of pos...
5056048,15_169,15,169,0,1404168000000,"RT @caughtoffside: #ALG gave it their all, was..."


In [132]:
# swifter.register_parallel_series_accessor(df['Tweet'])

In [133]:
del li

In [134]:
gc.collect()

2656

In [135]:
# Preprocessing of tweet
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^a-z\s]', '', text)
    # Remove numbers
    # text = re.sub(r'\d+', '', text)
    # Tokenization
    words = text.split()
    # Remove stopwords
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    # Lemmatization
    # words = [lemmatizer.lemmatize(word) for word in words]
    return ''.join(words)

In [136]:
# Apply preprocessing to each tweet
df['Tweet'] = df['Tweet'].swifter.apply(preprocess_text)
df

Pandas Apply: 100%|██████████| 5056050/5056050 [02:55<00:00, 28756.48it/s]


,ID,MatchID,PeriodID,EventType,Timestamp,Tweet
0,0_0,0,0,0,1276869000000,usastatesidefollowerstandrepresentbeautifulgam...
1,0_0,0,0,0,1276869000000,lynzthinkrefmightbasilfawltyactuallyworldcupger
2,0_0,0,0,0,1276869000000,hopingusawinhelpeasepainlastnightlossletgodemp...
3,0_0,0,0,0,1276869000000,actuallystartworldcup
4,0_0,0,0,0,1276869000000,hansonroyproperpunditlineworldcup
...,...,...,...,...,...,...
5056045,15_169,15,169,0,1404168000000,rtfoxsoccerworldcupquarterfinalsetgervfrafrida...
5056046,15_169,15,169,0,1404168000000,rtrodolphhilalplzguyretweetlettrendalgrespectf...
5056047,15_169,15,169,0,1404168000000,rtjoeybartonalgeriatakelotpositivebigbigheartw...
5056048,15_169,15,169,0,1404168000000,rtcaughtoffsidealggavewasntenoughhearttalentac...


In [137]:
# df = pd.read_csv('preprocessed_data.csv')

In [138]:
# df.drop(columns=['index'], inplace=True)

In [139]:
# df.reset_index(inplace=True)

In [140]:
# df.drop(columns=['Unnamed: 0'], inplace=True)

In [141]:
# key_i = [0]
keys = {}
tot_counts = {}
def get_ngram_counts(string: str, n=5,top=200, key_i=None):
    # string = str(string)
    counts = {}
    for i in range(len(string)-n):
        k = string[i:i+n]
        key = keys.setdefault(k, len(keys))
        tot_counts[key] = tot_counts.get(key, 0) + 1
        counts[key] = counts.get(key, 0) + 1
    return counts

In [142]:
df['counts'] = df['Tweet'].swifter.apply(get_ngram_counts, n=8)

Pandas Apply: 100%|██████████| 5056050/5056050 [03:22<00:00, 25003.34it/s]


In [143]:
df = df.drop(columns=['Tweet'])

In [144]:
df = df.drop(columns=['Timestamp'])

In [145]:
i_keys = {v: k for k, v in keys.items()}

In [146]:
top_200 = sorted(tot_counts.keys(), key=tot_counts.__getitem__, reverse=True)[:200]

In [147]:
# def filter_count(counts):    
#     return {k: v for k, v in counts.items() if k in top_200}

In [148]:
# df['counts'] = df['counts'].swifter.apply(filter_count)

In [149]:
def to_ndarr(counts):
    res = np.zeros((200,))
    for i, x in enumerate(top_200):
        res[i] = counts.get(x, 0)
    return res

In [150]:
df['counts'] = df['counts'].swifter.apply(to_ndarr)

Pandas Apply: 100%|██████████| 5056050/5056050 [02:56<00:00, 28618.63it/s]


In [151]:
df.head()

,ID,MatchID,PeriodID,EventType,counts
0,0_0,0,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,0_0,0,0,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,0_0,0,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,0_0,0,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,0_0,0,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


# Tweet Embeddings

In [152]:
# by now should have df with columns: ID, match id, period id, Event Type, tweet_vector. Tweet_vector is just 200 columns
df

,ID,MatchID,PeriodID,EventType,counts
0,0_0,0,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,0_0,0,0,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,0_0,0,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,0_0,0,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,0_0,0,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...
5056045,15_169,15,169,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5056046,15_169,15,169,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5056047,15_169,15,169,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5056048,15_169,15,169,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [153]:
# group the tweets into their corresponding periods to generate an average embedding vector for each period
# so there are no duplicate period id rows per match
# decreases size of data + makes it easier to fit into LSTM model
df = df.groupby(['MatchID', 'PeriodID', 'ID']).sum().reset_index()
df.drop(columns=['ID'], inplace=True) 
df['MatchID'] = df['MatchID'].astype(int)
df['PeriodID'] = df['PeriodID'].astype(int)
# need to convert to int before sorting
df.sort_values(by=['MatchID', 'PeriodID'], inplace=True)
df.reset_index(drop=True, inplace=True)


In [154]:
df.loc[df["EventType"] != 0, "EventType"] = 1
df

,MatchID,PeriodID,EventType,counts
0,0,0,0,"[142.0, 3.0, 2.0, 59.0, 1.0, 9.0, 0.0, 9.0, 1...."
1,0,1,0,"[170.0, 2.0, 1.0, 55.0, 0.0, 2.0, 0.0, 10.0, 1..."
2,0,2,0,"[152.0, 3.0, 3.0, 70.0, 0.0, 5.0, 0.0, 5.0, 1...."
3,0,3,1,"[154.0, 1.0, 1.0, 83.0, 1.0, 1.0, 0.0, 5.0, 0...."
4,0,4,0,"[169.0, 5.0, 4.0, 86.0, 0.0, 8.0, 0.0, 6.0, 0...."
...,...,...,...,...
2132,15,165,1,"[1403.0, 75.0, 75.0, 402.0, 13.0, 271.0, 38.0,..."
2133,15,166,1,"[1462.0, 54.0, 51.0, 332.0, 11.0, 235.0, 62.0,..."
2134,15,167,1,"[1551.0, 72.0, 67.0, 341.0, 14.0, 280.0, 115.0..."
2135,15,168,1,"[1436.0, 68.0, 66.0, 288.0, 12.0, 284.0, 116.0..."


In [155]:
largest_count = df.counts.swifter.apply(np.max).max()
largest_count

Pandas Apply: 100%|██████████| 2137/2137 [00:00<00:00, 244356.14it/s]


4202.0

In [180]:
# df['tweet_vector'] = df.counts.swifter.apply(lambda v: v / largest_count)
df['tweet_vector'] = df.tweet_vector.swifter.apply(lambda v: v / np.linalg.norm(v))

Pandas Apply: 100%|██████████| 2137/2137 [00:00<00:00, 146748.11it/s]


In [157]:
df.drop(columns=['counts'], inplace=True)

# Separate Train and Test data

In [181]:
# train on of the first 13 of 16 matches (16*0.8=12.8~=13)
# and the test data would be the last 3 matches. 
# Before submitting on Kaggle we should train on full dataset, so al 16 matches
train_percentage = 0.8
unique_match_ids = df['MatchID'].unique()
print(unique_match_ids)
num_matches_training = int(ceil(len(unique_match_ids)*train_percentage))
print(num_matches_training)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
13


In [182]:
target_match_id = num_matches_training
# target_match_id is first match id that will appear in test set
# all matches from target_match_id and after will be in test test
print(target_match_id)

13


In [183]:

#df2 = df['MatchID'] == 15
#df2

In [184]:
# row_index is first row with match id target_match_id
# row_index is then the first row of the matches that will go to the test


row_index = (df['MatchID'] == target_match_id).idxmax()
#row_index = df[df['MatchID'] == target_match_id].first_valid_index()
df_X_train = df[:row_index].copy()
df_X_test = df[row_index:].copy()


In [185]:
df_y_train = df_X_train['EventType']
df_y_test = df_X_test['EventType']

In [186]:
df_y_train

0       0
1       0
2       0
3       1
4       0
       ..
1702    1
1703    1
1704    1
1705    1
1706    1
Name: EventType, Length: 1707, dtype: int64

In [187]:
df_y_test.reset_index(drop=True, inplace=True)
df_y_test

0      0
1      0
2      1
3      1
4      1
      ..
425    1
426    1
427    1
428    1
429    0
Name: EventType, Length: 430, dtype: int64

In [188]:
df_X_train.drop(['EventType'], axis=1, inplace=True)
df_X_test.drop(['EventType'], axis=1, inplace=True)

In [189]:
df_X_train

,MatchID,PeriodID,tweet_vector
0,0,0,"[0.8494318028822557, 0.017945742314413853, 0.0..."
1,0,1,"[0.8619896147281682, 0.010141054290919625, 0.0..."
2,0,2,"[0.8294131404256828, 0.01636999619261216, 0.01..."
3,0,3,"[0.8018767131224206, 0.005206991643652082, 0.0..."
4,0,4,"[0.8060606905464752, 0.023847949424451925, 0.0..."
...,...,...,...
1702,12,125,"[0.7716263917507667, 0.0003496268200048784, 0...."
1703,12,126,"[0.7398922286824047, 0.00024338560154026466, 0..."
1704,12,127,"[0.720644187126446, 0.0, 0.0, 0.18271698742347..."
1705,12,128,"[0.7285631595334764, 0.0, 0.0, 0.1939886296416..."


In [190]:
df_X_test.reset_index(drop=True, inplace=True)
df_X_test

,MatchID,PeriodID,tweet_vector
0,13,0,"[0.054260354861444586, 0.0, 0.0, 0.00638357116..."
1,13,1,"[0.05511249262390522, 0.0, 0.0, 0.003131391626..."
2,13,2,"[0.06482748625538819, 0.0, 0.0, 0.008840111762..."
3,13,3,"[0.07454924713806146, 0.0, 0.0, 0.010303554482..."
4,13,4,"[0.07876198056923918, 0.0, 0.0, 0.010643510887..."
...,...,...,...
425,15,165,"[0.6765971432394946, 0.03616877102135573, 0.03..."
426,15,166,"[0.7385630187331415, 0.02727934542516391, 0.02..."
427,15,167,"[0.7439213427250922, 0.03453406620000428, 0.03..."
428,15,168,"[0.742239598496997, 0.03514783614052632, 0.034..."


In [191]:
# now df_X_train and df_X_test should have columns MatchID, PeriodID, tweet_vector. Tweet_vector is just 200 columns
# df_y_train and df_y_test should have 1 column, EventType
# the matchids are grouped together so all the rows of the same
# match ids are grouped next to each other, and the periodID are ordered chronologically.

In [192]:
# not working on my machine so I keep DF

# now we have df_X_train, df_X_test, df_y_train, df_y_test
# we no longer need df so we should free up the memory
# del df  # remove reference to the original DataFrame
# gc.collect()  # force garbage collection to free up memory

In [193]:
max_periods = df_X_train.groupby('MatchID')['PeriodID'].max().reset_index()
max_periods
# as we can see not every match has the same number of periods!

,MatchID,PeriodID
0,0,129
1,1,129
2,2,129
3,3,179
4,4,96
5,5,129
6,6,129
7,7,129
8,8,129
9,9,129


In [194]:
max_periods = df_X_test.groupby('MatchID')['PeriodID'].max().reset_index()
max_periods

,MatchID,PeriodID
0,13,129
1,14,129
2,15,169


# Format data for PyTorch LSTM

In [195]:
# input tensor for a PyTorch LSTM should have the shape of (when setting batch_first=True)
# (batch_size, seq_len, num_features) when using the batch_first=True parameter
# batch_size is number of sequences processed at once

# TRY WITHOUT SLIDING WINDOW APPROACH
#    which would mean batch size = number of matches
#    much easier to format for LSTM as 3D tensor
#    dimension of 3D tensor with batch_first=True:(batch_size = num_matches, seq_len = num_periods, num _features = 200)
#    (match_id, period_id, num_features=200)
#     not every match has the same number of periods!, so seq_len can vary between different matches
#     fix: will have to pad with zeroes
# we want tensor[match_id][period_id] to return list len 200 of corresponding tweet vector


In [196]:
df_X_train

,MatchID,PeriodID,tweet_vector
0,0,0,"[0.8494318028822557, 0.017945742314413853, 0.0..."
1,0,1,"[0.8619896147281682, 0.010141054290919625, 0.0..."
2,0,2,"[0.8294131404256828, 0.01636999619261216, 0.01..."
3,0,3,"[0.8018767131224206, 0.005206991643652082, 0.0..."
4,0,4,"[0.8060606905464752, 0.023847949424451925, 0.0..."
...,...,...,...
1702,12,125,"[0.7716263917507667, 0.0003496268200048784, 0...."
1703,12,126,"[0.7398922286824047, 0.00024338560154026466, 0..."
1704,12,127,"[0.720644187126446, 0.0, 0.0, 0.18271698742347..."
1705,12,128,"[0.7285631595334764, 0.0, 0.0, 0.1939886296416..."


In [197]:
def convert_df_to_3D_tensor(df_X, df_y):
    # df_X should have columns MatchID, PeriodID, tweet_vector. Tweet_vector is just 200 columns
    # rows with same matchID should be grouped together (adjacent rows)
    # df_y should have one column (the EventType)
    # returns tensor_X numpy array already padded! shape: (num_matches, max_num_periods, num _features = 200)
    # and tensor_y of shape: (num_matches, max_num_periods) 
    
    num_matches = len(df_X['MatchID'].unique())
    max_periods = df_X.groupby('MatchID')['PeriodID'].max().reset_index()
    total_max_period = max_periods['PeriodID'].max()
    #total_max_period is max seq len

    tensor_X = np.zeros((num_matches, total_max_period+1, 200))

    tensor_y = np.zeros((num_matches, total_max_period+1))
    print(tensor_X.shape)
    print(tensor_y.shape)
    
    i=0
    previous_match_id = df_X['MatchID'][0]
    for row_index, row in df_X.iterrows():
        match_id = int(row['MatchID'])

        if match_id != previous_match_id:
            i+=1
            previous_match_id = match_id
        
        period_id = int(row['PeriodID'])
        
        features = row.tweet_vector # Skip MatchID and PeriodID
        tensor_X[i, period_id, :] = features
        tensor_y[i,period_id] = df_y[row_index]
        
    return tensor_X, tensor_y

X_train_tensor, y_train_tensor = convert_df_to_3D_tensor(df_X_train, df_y_train)


(13, 180, 200)
(13, 180)


In [198]:
#print(X_train_tensor[0][3])
#print(y_train_tensor[0][3])

In [199]:
# SCALE DATA? minmaxscaler for example!
# SCALING MIGHT BE UNNECESSARY SINCE OUTPUT OF GLOVE TWEET 200 IS ALREADY SCALED BETWEEN -1 AND 1
#scaler = MinMaxScaler()
#tensor = scaler.fit_transform(tensor)

# CONVERT TO PYTORCH TENSOR
X_train_tensor = torch.tensor(X_train_tensor, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_tensor, dtype=torch.float32)

print(X_train_tensor.shape)
print(y_train_tensor.shape)
# X_train_tensor, y_train_tensor are now pytorch tensors

torch.Size([13, 180, 200])
torch.Size([13, 180])


# LSTM Model

In [200]:
# TODO VERIFY ITS CORRECT + MAKE MORE SOPHISTICATED
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout_rate):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out)
        out = out[:,:,:]
        out = self.sigmoid(out) # applying sigmoid to convert to probabilities
        return out.squeeze(-1)

#TODOOOOOOOOOO torch.nn.utils.rnn.pack_padded_sequence. This allows the model to ignore the padded values during computation.

In [201]:
batch_size = 180
batch_idxs = list(range(0,X_train_tensor.shape[0],batch_size))
batched_data = []
for idx in batch_idxs:
    batched_data.append((X_train_tensor[idx:idx+batch_size], y_train_tensor[idx:idx+batch_size]))

# Train model

In [214]:
hidden_size = 500 # can tune
num_layers = 4 # can tune
dropout_rate = 0.2 # can tune
num_epochs = 300 # can tune
lr = 0.001 # can tune

model = LSTMModel(input_size=200, hidden_size=hidden_size, num_layers=num_layers, dropout_rate=dropout_rate)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCELoss() # great for binary classification
#print(f"Shape of X_train_tensor: {X_train_tensor.shape}")
model.train()
for epoch in range(num_epochs):
    for train, label in batched_data:
        optimizer.zero_grad()
        outputs = model(train)
        #print(f"shape of outputs: {outputs.shape}")
        
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}")

print("Model is trained! (on training data)")

Epoch [0/300], Loss: 0.6890
Epoch [10/300], Loss: 0.5340
Epoch [20/300], Loss: 0.5111
Epoch [30/300], Loss: 0.4712
Epoch [40/300], Loss: 0.4454
Epoch [50/300], Loss: 0.4147
Epoch [60/300], Loss: 0.3775
Epoch [70/300], Loss: 0.4085
Epoch [80/300], Loss: 0.3651
Epoch [90/300], Loss: 0.3434
Epoch [100/300], Loss: 0.3225
Epoch [110/300], Loss: 0.3071
Epoch [120/300], Loss: 0.2912
Epoch [130/300], Loss: 0.2657
Epoch [140/300], Loss: 0.2678
Epoch [150/300], Loss: 0.2582
Epoch [160/300], Loss: 0.2227
Epoch [170/300], Loss: 0.2096
Epoch [180/300], Loss: 0.1989
Epoch [190/300], Loss: 0.1880
Epoch [200/300], Loss: 0.1554
Epoch [210/300], Loss: 0.1581
Epoch [220/300], Loss: 0.2365
Epoch [230/300], Loss: 0.1257
Epoch [240/300], Loss: 0.0963
Epoch [250/300], Loss: 0.0840
Epoch [260/300], Loss: 0.1096
Epoch [270/300], Loss: 0.2192
Epoch [280/300], Loss: 0.1476
Epoch [290/300], Loss: 0.1043
Model is trained! (on training data)


# Evaluate on test data

In [215]:
# convert df_X_test and df_y_test to correct format/dimensions
X_test_tensor, y_test_tensor = convert_df_to_3D_tensor(df_X_test, df_y_test)
# CONVERT TO PYTORCH TENSOR
X_test_tensor = torch.tensor(X_test_tensor, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_tensor, dtype=torch.float32)

(3, 170, 200)
(3, 170)


In [216]:
#print(X_test_tensor[2][129])
#print(y_test_tensor[2][129])

In [217]:




model.eval()

with torch.no_grad():
    predictions = model(X_test_tensor)

# predictions have values between 0 and 1 because forward pass of LSTM contains sigmoid at output
#print(predictions)

predicted_classes = (predictions > 0.5).float() # 0.5 is threshold
#this converts to same dimensional array of True or false, and .float() converts True to 1 and False to 0

#print(predicted_classes)
 

In [218]:
predicted_classes

tensor([[0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0.,
         0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1.,
         0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1.,
         0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0.,
         0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.

In [219]:
#print(predictions.shape)

In [220]:
# performance metrics
from random import randint

loss = criterion(predictions, y_test_tensor) # use predictions for loss calculation

print(f"Binary Cross-Entropy Loss: {loss.item():.4f}")

def accuracy(y_true, y_pred):
    if y_true.dtype != y_pred.dtype or y_true.shape != y_pred.shape:
        raise ValueError(f"Inputs do not have same type or shape!")
    y_rand = torch.randint(0,2,y_true.shape)
    correct_predictions = (y_true == y_pred).sum().item()
    total_predictions = y_true.numel()
    accuracy = correct_predictions / total_predictions * 100
    return accuracy
accuracy = accuracy(y_test_tensor, predicted_classes)


print(f"Accuracy: {accuracy:.4f}")

#print(y_test_tensor.shape)
#print(predicted_classes.shape)


# Visualization of Actual vs Predicted Classes
# import matplotlib.pyplot as plt
# TODO COULD USE PLT TO VISUALIZE?

Binary Cross-Entropy Loss: 1.3348
Accuracy: 70.9804


# For Kaggle Submission

In [221]:
# RETRAIN MODEL ON ENTIRE TRAINING DATA AND EVALUATE EVAL TWEETS




df_X = pd.concat([df_X_train, df_X_test], ignore_index=True)
df_y = pd.concat([df_y_train, df_y_test], ignore_index=True)




In [222]:
df_X

,MatchID,PeriodID,tweet_vector
0,0,0,"[0.8494318028822557, 0.017945742314413853, 0.0..."
1,0,1,"[0.8619896147281682, 0.010141054290919625, 0.0..."
2,0,2,"[0.8294131404256828, 0.01636999619261216, 0.01..."
3,0,3,"[0.8018767131224206, 0.005206991643652082, 0.0..."
4,0,4,"[0.8060606905464752, 0.023847949424451925, 0.0..."
...,...,...,...
2132,15,165,"[0.6765971432394946, 0.03616877102135573, 0.03..."
2133,15,166,"[0.7385630187331415, 0.02727934542516391, 0.02..."
2134,15,167,"[0.7439213427250922, 0.03453406620000428, 0.03..."
2135,15,168,"[0.742239598496997, 0.03514783614052632, 0.034..."


In [223]:
df_y

0       0
1       0
2       0
3       1
4       0
       ..
2132    1
2133    1
2134    1
2135    1
2136    0
Name: EventType, Length: 2137, dtype: int64

In [224]:
# convert df_X_test and df_y_test to correct format/dimensions
X_tensor, y_tensor = convert_df_to_3D_tensor(df_X, df_y)
# CONVERT TO PYTORCH TENSOR
X_tensor = torch.tensor(X_tensor, dtype=torch.float32)
y_tensor = torch.tensor(y_tensor, dtype=torch.float32)

(16, 180, 200)
(16, 180)


In [225]:
# NOTES
# HOW TO MAKE SURE THAT we:
# 1. DO NOT ignore the order of the tweets -> (LSTM)
# 2. treat each time period as RELATED to the football match they belong to -> treat each match as a sequence, train LSTM on every sequence
#                      since pytorch tensor expects multiple sequences (batches)



# for LSTM: Each input sequence should consist of tweets from a specific match, ordered by Period ID.
#   tweets of different matches are unrelated, but tweets of a same match are related sequentially (chronologically)
#   structure training data such that tweets are grouped by match id, and ordered by period id